In [1]:
import pandas as pd
import os.path as op
from os import sep
import nibabel as nb
import numpy as np
import json
import trimesh
import open3d as o3d
import open3d.visualization.rendering as rendering
import matplotlib.pylab as plt
from matplotlib import cm, colors
from utilities import files
import new_files
import tqdm.auto as tqdm
from copy import copy
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler, minmax_scale
from sklearn.manifold import MDS
from scipy.spatial.distance import euclidean
from brain_tools import *
import pickle
from functools import reduce

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def plot_csd(smooth_csd, list_ROI_vertices, bb_path, times, ax, cb=True, cmap="RdBu_r", vmin_vmax=None):
    layer_labels = ["I", "II", "III", "IV", "V", "VI"]
    with open(bb_path, "r") as fp:
        bb = json.load(fp)
    bb = [np.array(bb[i])[list_ROI_vertices] for i in bb.keys()]
    bb_mean = [np.mean(i) for i in bb]
    bb_std = [np.std(i) for i in bb]
    max_smooth = np.max(np.abs(smooth_csd))
    if vmin_vmax == None:
        divnorm = colors.TwoSlopeNorm(vmin=-max_smooth, vcenter=0, vmax=max_smooth)
    else:
        divnorm = colors.TwoSlopeNorm(vmin=vmin_vmax[0], vcenter=0, vmax=vmin_vmax[1])
    extent = [times[0], times[-1], 1, 0]
    csd_imshow = ax.imshow(
        smooth_csd, norm=divnorm, origin="lower",
        aspect="auto", extent=extent,
        cmap=cmap
    )
    ax.set_ylim(1,0)
    for l_ix, th in enumerate(np.cumsum(bb_mean)):
            ax.axhline(th, linestyle=(0, (5,5)), c="black", lw=0.5)
            # ax.axhspan(th-bb_std[l_ix], th+bb_std[l_ix], alpha=0.05, color="black", lw=0)
            ax.annotate(layer_labels[l_ix],[times[0]+0.01, th-0.01],size=15)
    if cb:
        plt.colorbar(csd_imshow, ax=ax)
    plt.tight_layout()

In [3]:
dataset_location = "/home/common/bonaiuto/multiburst/derivatives/processed"
epoch_types = {
    "visual": [np.linspace(-0.2, 0.8, num=601), [0.0, 0.2], -0.01],
    "motor": [np.linspace(-0.5, 0.5, num=601), [-0.2, 0.2], -0.2]
}

In [5]:
csd_files = dir_search.get_files(
    dataset_location, "*.npy", prefix="time_CSD_autoreject"
)

json_files = dir_search.get_files(
    dataset_location, "*.json", prefix="info"
)

info_dict = {}
for i in json_files:
    sub = i.split(sep)[-3]
    with open(i, "r") as fp:
        info_dict[sub] = json.load(fp)

In [6]:
outlier_maps = {} # core_name, roi
pca_results = {} # core_name, roi
pca_prc_csd = {} # core_name, roi, pc_comp
sc_log_all = {} # core_name, roi, pc_comp

for csd_file in tqdm.tqdm(csd_files):
    epoch_type = [i for i in epoch_types.keys() if i in csd_file][0]
    subject = csd_file.split(sep)[-4]
    core_name = csd_file.split(sep)[-1].split("_")[-1].split(".")[0]
    info = info_dict[subject]
    atlas = pd.read_csv(info["atlas"])
    atlas_labels = np.load(info["atlas_colors_path"])
    visual_ROI = atlas.loc[(atlas.PRIMARY_SECTION == 1)].USED_LABEL.values
    visual_ROI = np.hstack([visual_ROI, [i for i in atlas.USED_LABEL.values if "_MT_" in i]])
    sensorimotor_ROI = ["L_4_ROI", "R_4_ROI"]
    labels_xxx = {
        "visual": visual_ROI,
        "motor": sensorimotor_ROI
    }
    ROI_labels = labels_xxx[epoch_type]
    vertex_num = np.arange(atlas_labels.shape[0])
    ROI_vertices = {i: vertex_num[[i == al.decode("utf=8") for al in atlas_labels]] for i in ROI_labels}
    times, pca_sel, baseline_lim = epoch_types[epoch_type]

    csd_data = np.load(csd_file)
    true_CSD = {}
    for l in ROI_labels:
        true_CSD[l] = []
        for rv in ROI_vertices[l]:
            true_CSD[l].append(csd_data[rv, :, :])
    true_CSD = {i: np.array(true_CSD[i]) for i in ROI_labels}
    pca_time_sel = np.where((times >= pca_sel[0]) & (times <= pca_sel[1]))[0]
    pca_csd_dataset = {i: true_CSD[i][:,:, pca_time_sel].reshape(true_CSD[i].shape[0], -1) for i in ROI_labels}
    
    for roi_ix, roi in enumerate(ROI_labels):
        metric = pca_csd_dataset[roi].std(axis=1)
        nan_map = np.isnan(metric)
        minmax = np.percentile(metric[~nan_map], 0.005), np.percentile(metric[~nan_map], 99.995)
        outlier_map = metric > minmax[1] + minmax[1] * 0.000
        unwanted_map = nan_map | outlier_map
        outlier_maps[(core_name, roi)] = unwanted_map

    for roi_ix, roi in enumerate(ROI_labels):
        ds = pca_csd_dataset[roi][~outlier_maps[(core_name, roi)]]
        scaler = RobustScaler()
        ds = scaler.fit_transform(ds)
        if pca_csd_dataset[roi].shape[0] < 30:
            pca = PCA(n_components=pca_csd_dataset[roi].shape[0] - 2)
        else:
            pca = PCA(n_components=30)
        ds_pca = pca.fit_transform(ds)
        pca_results[(core_name, roi)] = ds_pca, pca.components_, pca.explained_variance_ratio_

    SMOOTH_CSD = {}
    for l in ROI_labels:
        SMOOTH_CSD[l] = []
        for rv in ROI_vertices[l]:
            SMOOTH_CSD[l].append(smooth_csd(csd_data[rv, :, :], info["n_surf"]))
    for l in ROI_labels:
        SMOOTH_CSD[l] = np.array(SMOOTH_CSD[l])[~outlier_maps[(core_name, l)]]

    prc = np.linspace(0, 100, num=21)
    prc_bounds = list(zip(prc[:-1], prc[1:]))
    n_bins = 100
    for l in ROI_labels:
        for pc_comp in range(4):
            sc_log = np.log10(np.abs(pca_results[(core_name, l)][0][:, pc_comp]))
            pc_score = pca_results[(core_name, l)][0][:, pc_comp]
            csd_bounds = []
            for ix, pb in enumerate(prc_bounds):
                bounds = [np.percentile(pc_score, i) for i in pb]
                pr_mask = np.where((pc_score >= bounds[0]) & (pc_score < bounds[1]))[0]
                mean_smooth_csd = np.mean(SMOOTH_CSD[l][pr_mask], axis=0)
                baseline = np.mean(mean_smooth_csd[:, np.where(times < baseline_lim)], axis=2)
                csd_bounds.append(mean_smooth_csd - baseline)
            csd_bounds = np.array(csd_bounds)
            pca_prc_csd[(core_name, l, pc_comp)] = csd_bounds
            sc_log_all[(core_name, l, pc_comp)] = sc_log

  0%|          | 0/12 [00:00<?, ?it/s]

In [7]:
for csd_file in tqdm.tqdm(csd_files):
    core_name = csd_file.split(sep)[-1].split("_")[-1].split(".")[0]
    epoch_type = [i for i in epoch_types.keys() if i in core_name][0]
    subject = csd_file.split(sep)[-4]
    info = info_dict[subject]
    atlas = pd.read_csv(info["atlas"])
    atlas_labels = np.load(info["atlas_colors_path"])
    visual_ROI = atlas.loc[(atlas.PRIMARY_SECTION == 1)].USED_LABEL.values
    visual_ROI = np.hstack([visual_ROI, [i for i in atlas.USED_LABEL.values if "_MT_" in i]])
    sensorimotor_ROI = ["L_4_ROI", "R_4_ROI"]
    labels_xxx = {
        "visual": visual_ROI,
        "motor": sensorimotor_ROI
    }
    ROI_labels = labels_xxx[epoch_type]
    vertex_num = np.arange(atlas_labels.shape[0])
    ROI_vertices = {i: vertex_num[[i == al.decode("utf=8") for al in atlas_labels]] for i in ROI_labels}
    times, pca_sel, baseline_lim = epoch_types[epoch_type]
    time_sel = np.where((times >= pca_sel[0]) & (times <= pca_sel[1]))[0]

    for pc_comp in range(4):
        rows = len(ROI_labels)
        f, ax = plt.subplots(rows, 3, figsize=(14, 4*rows), facecolor="white")
        ax[0,0].set_title("{} - {} percentile".format(*prc_bounds[0]))
        ax[0,1].set_title("{} - {} percentile".format(*prc_bounds[-1]))
        ax[0,2].set_title("eigenvector motifs")
        for row, roi in enumerate(ROI_labels):
            lab_row = atlas.loc[atlas.USED_LABEL == roi]
            label = "{} {}".format(lab_row.HEMISPHERE.values[0], lab_row.LONG_NAME.values[0])
            ax[row, 0].set_ylabel(label)
            vmm = pca_prc_csd[core_name, roi, pc_comp].min(), pca_prc_csd[core_name, roi, pc_comp].max()
            data = np.array([pca_prc_csd[core_name, roi, pc_comp][0], pca_prc_csd[core_name, roi, pc_comp][-1]])
            # vmm = np.percentile(data, 5), np.percentile(data, 95)
            plot_csd(data[0], ROI_vertices[roi], info["big_brain_layers_path"], times, ax=ax[row,0])
            plot_csd(data[1], ROI_vertices[roi], info["big_brain_layers_path"], times, ax=ax[row,1])
            ev = smooth_csd(np.array(np.split(pca_results[(core_name, roi)][1][pc_comp], info["layers"])), info["layers"])
            nan_ar = np.zeros(data[0].shape)
            nan_ar[nan_ar == 0] = np.nan
            nan_ar[:, time_sel] = ev
            plot_csd(nan_ar, ROI_vertices[roi], info["big_brain_layers_path"], times, ax=ax[row,2], cmap="viridis")
            for col in range(3):
                ax[row, col].axvline(times[time_sel][0], lw=0.2, c="black")
                ax[row, col].axvline(times[time_sel][-1], lw=0.2, c="black")
        f.suptitle("PC {} | {}".format(pc_comp+1, core_name), y=1.05);
        plt.savefig("/home/mszul/git/DANC_multilayer_laminar/output/{}_PC_{}_CSD.png".format(core_name, str(pc_comp+1).zfill(2), l), dpi=300)
        plt.close(f)

  0%|          | 0/12 [00:00<?, ?it/s]

In [10]:
brain_render = {}
mpl_scores = {}
sc_log_scores = {}

for csd_file in tqdm.tqdm(csd_files):
    core_name = csd_file.split(sep)[-1].split("_")[-1].split(".")[0]
    epoch_type = [i for i in epoch_types.keys() if i in core_name][0]
    subject = csd_file.split(sep)[-4]
    info = info_dict[subject]
    atlas = pd.read_csv(info["atlas"])
    atlas_labels = np.load(info["atlas_colors_path"])
    visual_ROI = atlas.loc[(atlas.PRIMARY_SECTION == 1)].USED_LABEL.values
    visual_ROI = np.hstack([visual_ROI, [i for i in atlas.USED_LABEL.values if "_MT_" in i]])
    sensorimotor_ROI = ["L_4_ROI", "R_4_ROI"]
    labels_xxx = {
        "visual": visual_ROI,
        "motor": sensorimotor_ROI
    }
    ROI_labels = labels_xxx[epoch_type]
    vertex_num = np.arange(atlas_labels.shape[0])
    ROI_vertices = {i: vertex_num[[i == al.decode("utf=8") for al in atlas_labels]] for i in ROI_labels}
    rows = len(ROI_labels)

    gray = np.array([0.5, 0.5, 0.5])
    brain = nb.load(info["pial_ds_nodeep_inflated"])
    vertices, faces = brain.agg_data()
    colours = np.repeat(gray.reshape(1,-1), vertices.shape[0], axis=0)
    
    f, ax = plt.subplots(rows, 1, figsize=(20, 4*rows), facecolor="white")
    for row, roi in enumerate(ROI_labels):
        lab_row = atlas.loc[atlas.USED_LABEL == roi]
        label = "{} {}".format(lab_row.HEMISPHERE.values[0], lab_row.LONG_NAME.values[0])
        sc_log = []
        ax[row].set_ylabel(label)
        for pc_comp in range(4):
            sc_log.append(sc_log_all[(core_name, roi, pc_comp)])
        mpl_score = np.multiply(np.multiply(sc_log[0], sc_log[1]), np.multiply(sc_log[2], sc_log[3]))
        n_bins = 100
        datacolors, mappable = data_to_rgb(
            mpl_score, n_bins, "pink_r",
            vmin=np.percentile(mpl_score, 5),
            vcenter=np.percentile(mpl_score, 50),
            vmax=np.percentile(mpl_score, 95),
            ret_map=True
        )
        
        hist, bins, barlist = ax[row].hist(mpl_score, bins=n_bins, edgecolor='black', linewidth=0.2)
        for ix, xx in enumerate(barlist):
            plt.setp(xx, "facecolor", mappable.to_rgba(bins[ix+1]))

        data_colour_map = np.repeat(np.array([[0, 1, 0.224]]), ROI_vertices[roi].shape[0], axis=0)
        data_colour_map[~outlier_maps[core_name, roi]] = datacolors[:,:3]
        for ix, v in enumerate(ROI_vertices[roi]):
            colours[v] = data_colour_map[ix][:3]
        mpl_scores[(core_name, roi)] = mpl_score
        sc_log_scores[(core_name, roi)] = sc_log
    plt.savefig("/home/mszul/git/DANC_multilayer_laminar/output/{}_pc_score_multiplied.png".format(core_name, str(pc_comp+1).zfill(2), l), dpi=300)
    plt.close(f)
    
    
    brain_render[(subject, epoch_type, core_name)] = [core_name, info["pial_ds_nodeep_inflated"], colours]

  0%|          | 0/12 [00:00<?, ?it/s]

In [9]:
for k in brain_render.keys():
    core_name, brain_path, colours = brain_render[k]
    brain = nb.load(brain_path)
    vertices, faces = brain.agg_data()
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False, validate=False)
    mesh = mesh.as_open3d
    mesh.compute_vertex_normals(normalized=True)
    mesh.vertex_colors = o3d.utility.Vector3dVector(colours)
    # vxmask = np.array([i.decode("utf=8")[0] != roi[0] for i in atlas_labels]) | np.array([i.decode("utf=8")[0] == "?" for i in atlas_labels])
    # mesh.remove_vertices_by_mask(vxmask)
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
brain_render.keys()

In [ ]:
core_name, brain_path, colours = brain_render[('sub-001', 'motor', 'autoreject-sub-001-ses-01-001-motor-epo')]
brain = nb.load(brain_path)
vertices, faces = brain.agg_data()
mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False, validate=False)
mesh = mesh.as_open3d
mesh.compute_vertex_normals(normalized=True)
mesh.vertex_colors = o3d.utility.Vector3dVector(colours)
# vxmask = np.array([i.decode("utf=8")[0] != roi[0] for i in atlas_labels]) | np.array([i.decode("utf=8")[0] == "?" for i in atlas_labels])
# mesh.remove_vertices_by_mask(vxmask)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [12]:
with open("brain_render.pickle","wb") as f:
    pickle.dump(brain_render, f, protocol=0)